### RAG Piplines: Data Ingestion to Vector DB Pipeline

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm')

In [ ]:
### Read all the pdf's inside the directory
from pathlib import Path
def process_all_pdfs(pdf_directory):
  """Process all PDF files in a directory"""
  all_documents = []
  pdf_dir = Path(pdf_directory)

  # Find all PDF files recursively
  pdf_files = list(pdf_dir.glob("**/*.pdf"))
  print(f"Found {len(pdf_files)} PDF files to process.")

  for pdf_file in pdf_files:
    print(f"\nProcessing: {pdf_file.name}")
    try:
      loader = PyPDFLoader(str(pdf_file))
      documents = loader.load()

      # Add source information to metadata
      for doc in documents:
        doc.metadata['source_file'] = pdf_file.name
        doc.metadata['file_type'] = 'pdf'

        all_documents.extend(documents)
        print(f"Loaded {len(documents)} pages")

    except Exception as e:
      print(f"Error: {e}")

  print(f"\nTotal documents loaded: {len(all_documents)}")
  return all_documents

# Process all PDFs in the data directory

all_pdf_documents = process_all_pdfs("../data")


In [ ]:
all_pdf_documents

In [ ]:
### Text splitting get into chunks

def split_documents(documents, chunk_size=1000,chunk_overlap=200):
  """Split documents into smaller chunks for better RAG performance"""
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    separators=["\n\n","\n"," ",""]
  )
  split_docs = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

  if split_docs:
    print(f"\nExample chunk:")
    print(f"Content: {split_docs[0].page_content[:200]}...")
    print(f"Metadata: {split_docs[0].metadata}")

  return split_docs

In [ ]:
chunks=split_documents(all_pdf_documents)
chunks

### Step-3: Embeddings

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class EmbeddingManager:
  """
    Handles document embedding generation usng SentenceTransformer
  """
  def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
    """
    Initialize the embedding manager

    Args:
      model_name: HuggingFace model name for sentence embeedings
    """
    self.model_name = model_name
    self.model = None
    self._load_model()
  
  def _load_model(self):
    """Load the SentenceTransformer model"""
    try:
      print(f"Loading embedding model: {self.model_name}")
      self.model = SentenceTransformer(self.model_name)
      print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
    except Exception as e:
      print(f"Error loading model {self.model_name}: {e}")
      raise

  def generate_embeddings(self, texts: List[str]) -> np.ndarray:
    """
    Generate embeddings for a list of texts

    Args: 
      texts: List of text strings to emded
    
    Returns:
      numpy array of emdeddings with shape (len(texts), embedding_dim)

    """

    if not self.model:
      raise ValueError("Model not loaded")
    
    print(f"Generating emdeddings for {len(texts)} texts....")
    embeddings = self.model.encode(texts, show_progress_bar=True)
    print(f"Generated embeddings with shape: {embeddings.shape}")
    return embeddings
  
  def get_embedding_dimension(self) -> int:
    """Get the embedding dimension of the model"""
    if not self.model:
      raise ValueError("Model not loaded.")
    
    return self.model.get_sentence_embedding_dimension()
  

## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager  

### Step 4: VectorStore

In [ ]:
class VectorStore:
  """
    Manages document embedding in a ChromaDB vector store
  """
  def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
    """
    Initialize the vector store

    Args:
      collection_name: Name of the ChromaDB collection
      persist_directory: Directory to persist the vector store
    """
    self.collection_name = collection_name
    self.persist_directory = persist_directory
    self.client = None
    self.collection = None
    self._initialize_store()
  
  def _initialize_store(self):
    """Initialize ChromaDB client and collection"""

    try:
      # Create persistent ChromaDB client
      os.makedirs(self.persist_directory, exist_ok=True)
      self.client = chromadb.PersistentClient(path=self.persist_directory)

      # Get or create collection
      self.collection = self.client.get_or_create_collection(
        name=self.collection_name,
        metadata={"description": "PDF document embeddings for RAG"}
      )
      print(f"Vector store initialized. Collection: {self.collection_name}")
      print(f"Existing documents in collection: {self.collection.count()}")
    except Exception as e:
      print(f"Error initializing vector store: {e}")
      raise
  
  def add_documents(self, documents: List[Any], embeddings: np.array):
    """
    Add documents and their embeddings to the vector store
    
    Args:
      documents: List of LangChain documents
      embeddings: Corresponding embeddings for the documents
    """

    if len(documents) != len(embeddings):
      raise ValueError("Number of documents must match number of embeddings")
    
    print(f"Adding {len(documents)} documents to vector store...")

    # Prepare data for ChromaDB

    ids = []
    metadatas = []
    documents_text = []
    embeddings_list = []

    for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
      # Generate unique ID
      doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
      ids.append(doc_id)

      # Prepare metadata
      metadata = dict(doc.metadata)
      metadata['doc_index'] = i
      metadata['content_length'] = len(doc.page_content)
      metadatas.append(metadata)

      # Document content
      documents_text.append(doc.page_content)

      # Embedding

      embeddings_list.append(embedding.tolist())
    
    # Add to collection
    try:
      self.collection.add(
        ids=ids,
        embeddings=embeddings_list,
        metadatas=metadatas,
        documents=documents_text
      )
      print(f"Sucessfully added {len(documents)} documents to vector store")
      print(f"Total documents in collection: {self.collection.count()}")
    except Exception as e:
      print(f"Error adding documents to vector store: {e}")
      raise


# Initialize Vector Store
vectorstore = VectorStore()
vectorstore


In [ ]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings = embedding_manager.generate_embeddings(texts)

## Store in the vector database
vectorstore.add_documents(chunks, embeddings)


### Step 5: Retriever Pipeline From VectorStore

In [ ]:
class RAGRetriever:
  """Handles query-based retrival from the vector store"""

  def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
    """
    Initialize the retriever

    Args:
      vector_store: Vector store containing document embeddings
      embedding_manager: Manager for gernerating query embeddings
    """
    self.vector_store = vector_store
    self.embedding_manager = embedding_manager
    
  def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
    """
    Retrieve relevant documents for a query

    Args:
      query: The search query
      top_k: Number of top results to return
      score_threshold: Minimum similarity score threshold
    Returns:
      List of dictionaries containing retrieved documents and metadata
    """
    print(f"Retrieving documents for query: '{query}'")
    print(f"Top K: {top_k}, Score threshold: {score_threshold}")

    # Generate query embedding
    query_embedding = self.embedding_manager.generate_embeddings([query])[0]

    # Search in vector store

    try:
      results = self.vector_store.collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=top_k
      )

      # Processed results
      retrieved_docs = []

      if results['documents'] and results['documents'][0]:
        documents = results['documents'][0]
        metadatas = results['metadatas'][0]
        distances = results['distances'][0]
        ids = results['ids'][0]

        for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
          # Convert disance to similarity score (ChromaDB uses cosine distance)
          similarity_score = 1 - distance

          if similarity_score >= score_threshold:
            retrieved_docs.append({
              'id': doc_id,
              'content': document,
              'metadata': metadata,
              'similarity_score': similarity_score,
              'rank': i + 1
            })

            print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
          else:
            print(f"No documents found")
        return retrieved_docs
    except Exception as e:
      print(f"Error during retrieval: {e}")
      return []
    

rag_retriever = RAGRetriever(vectorstore, embedding_manager)

In [ ]:
rag_retriever

In [ ]:
rag_retriever.retrieve("What is Deep learning is all you need?")


In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv()

# Retrieve the key
api_key = os.getenv("OPENAI_API_KEY")
api_key

In [ ]:
# Initialize the Open AI LLM 
chat_model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0.1, max_completion_tokens=1024)

# Simple RAG function: retrieve context + generate response

def rag_simple(query, retriever, llm, top_k=3):
  # Retrieve relevant documents
  results = retriever.retrieve(query, top_k=top_k)
  # Combile retrieved document contents
  context = "\n\n".join([doc['content'] for doc in results]) if results else ""

  if not context:
    return "No relevant context found to answer the question."
  
  # Create prompt
  prompt = f"""Use the following context to answer concisely.
               Context: {context}
               Question: {query}
               Answer:
            """

  # Generate the answer using OpenAI LLM
  response = llm.invoke([prompt.format(context=context,query=query)])
  return response.content



In [ ]:
answer =rag_simple("What is deep learning?", rag_retriever, chat_model)
answer


### Step 6: Enhanched RAG Pipeline  Features

In [ ]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is Deep Learning?", rag_retriever, chat_model, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])



### Advanced RAG Pipeline: Streaming, Citations, History, Summarization

In [ ]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, chat_model)
result = adv_rag.query("what is deep learning is all you need", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])